In [ ]:
import sys, os, math
import numpy as np 

Welcome to tutorial one of this GMAT navigation series. In the following series of tutorials, you will see how to interact with the GMAT Python API. Moreover, at completion of this tutorial series, you should be well equipt to exploit the GMAT API to build estimation algorithms, such as the extended Kalman filter, the unscented Kalman filter, a particle filter, or truly, any general estimation algorithm for an application of your choosing.

Some notes to start: This series assumes that you have installed GMAT, its python wrapper, and also have some basic Python skills. Visit https://github.com/ChristopherRabotin/GMAT/tree/GMAT-2020a or your favorite GMAT fork for GMAT installation instructions.

In this tutorial, we introduce loading in the GMAT environment. System paths are required to locate GMAT, a 'Space Weather' file, and an 'Earth Orientation Parameters' (EOP) file. I've included SpaceWeather and EOP files for you in the subdirectory entitled "data", so dont fret. Change these to your mission standard, if you need.

The following code snippet checks for the above requirements. If you can run the following error free, you're set to use Python GMAT and continue onwards in this tutorial series. 

In each tutorial jupyter notebook, you will need to modify the section of code below the comment
>  ### --------------- USER DEFINED PATHS (USER -- PLEASE LOOK HERE!) ------------------
Go ahead and do so below, and then run the cell.

In [ ]:

# Directory location of this Jupyter notebook tutorial
file_dir = os.path.abspath('') #os.path.dirname(os.path.abspath(__file__))

# --------------- USER DEFINED PATHS (USER -- PLEASE LOOK HERE!) ------------------# 
# Directory of the GMAT folder -- You WILL NEED to change this!
gmat_root_dir = '/home/natsubuntu/Desktop/SysControl/estimation/CauchyCPU/CauchyEst_Nat/GMAT'
# Directory location of the SpaceWeather and EOP File -- You MAY NOT need to change this!
gmat_data_dir = file_dir + "/data"
# Name of EOP file and SpaceWeather file, which are located in gmat_data_dir -- You MAY NOT need to change these!
EOP_filename = "eop_file.txt"
SpaceWeather_filename = "SpaceWeather-v1.2.txt"

# --------------- AUTOMATED CHECKS ON USER CONFIGURED PATHS ----------------- #
# 1.) Test directory location of the SpaceWeather and EOP File for existence
if not os.path.isdir(gmat_data_dir):
    print("ERROR: gmat_root_dir: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_data_dir))
    exit(1)
gmat_eop_file = gmat_data_dir + "/" + EOP_filename
gmat_sw_file = gmat_data_dir + "/" + SpaceWeather_filename
if not os.path.isfile(gmat_eop_file):
    print("ERROR: gmat_eop_file: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_eop_file))
    exit(1)
if not os.path.isfile(gmat_sw_file):
    print("ERROR: gmat_sw_file: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_sw_file))
    exit(1)
# 2.) Check GMAT Subdirectories for existence, leaving nothing to chance
gmat_application_dir = gmat_root_dir + '/application' 
if os.path.exists(gmat_application_dir):
    # GMAT's bin, api, and python interface (gmatpy) directories, and the location of api_startup_file.txt, respectively
    gmat_bin_dir = gmat_application_dir + '/bin'
    gmat_api_dir = gmat_application_dir + '/api'
    gmat_py_dir = gmat_bin_dir + '/gmatpy'
    gmat_startup_file = gmat_bin_dir + '/api_startup_file.txt'
    if os.path.exists(gmat_bin_dir):
        if gmat_bin_dir not in sys.path:
            sys.path.append(gmat_bin_dir)
    else:
        print("ERROR: gmat_bin_dir: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_bin_dir))
        exit(1)
    if os.path.exists(gmat_api_dir):
        if gmat_api_dir not in sys.path:
            sys.path.append(gmat_api_dir)
    else:
        print("ERROR: gmat_api_dir: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_api_dir))
        exit(1)
    if os.path.exists(gmat_py_dir):
        if gmat_py_dir not in sys.path:
            sys.path.append(gmat_py_dir)
    else:
        print("ERROR: gmat_py_dir: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_py_dir))
        exit(1)
    if os.path.exists(gmat_startup_file):
        import gmat_py as gmat # The main act
        gmat.Setup(gmat_startup_file) # The main act
    else:
        print("ERROR: gmat_startup_file: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_startup_file))
        exit(1)
else:
    print("ERROR: gmat_application_dir: {}\nSEEN NOT TO EXIST! Please check path! Exiting!".format(gmat_application_dir))
    exit(1)
print("All Looks Good! Youre Set!")

If you got the message "All Looks Good! Youre Set!" above, then you're all setup and ready to use this tutorial series.

A pedantic rant: Its important to follow all installation steps for GMAT, as some files dont exist until built. For example, once you have installed GMAT, compiled, and built its Python wrapper, you will need to change directories to GMAT/application/api and execute 

> YOUR-PYTHON-VERSION BuildApiStartupFile.py

For example, using Python 3.7, one could do so as:
> python3.7 BuildApiStartupFile.py

This will place the api_startup_file.txt into the application/bin folder. This may give you a little insight into why we need to define the paths above.

As we go through these tutorials, we will refactor code blocks into a usable library to reduce our code redundancy. This will be pointed out after various tutorial checkpoints. All refactored code is located in "gmat_nav.py", located within this directory. The contents of "gmat_nav.py" provides functionality for most of your basic estimation/navigation needs, but by no means is a complete estimation library: its a jumping off point. For example, the above "check then load gmatpy" code block has been refactored into the function   
> check_then_load_gmatpy(gmat_root_dir, eop_filepath, spaceweather_filepath)
located in gmat_nav.py.

Last thing: if you are using an interactive debugger, such as vscode when building your own application (or viewing this tutorial series), it will be very helpful to have intellisense as you poke around the 'gmat_py' module (this I highly recommend). To give yourself intellisense with the gmat_py module, you can softlink it in the current directory:

For example:
> ln -s PATH_TO_GMATPY/gmat_py.py gmat_py.py 

will give you a softlink to GMAT Python in your current directory. Now, as you type, the various gmat_py wrapper functions will pop up. Its a very nice extra.